# Imports

In [2]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [3]:
from nim.NimGame import NimGame
from nim.NimLogic import NimLogic

from agents.MinimaxAgent import MinimaxAgent

# Setup

In [4]:
np.random.seed(42)

EPISODES = 1000
MAX_DEPTH = 9999

# Agent Testing Function

In [5]:
def test_agent(_misereAgent, _normalAgent, _misere, _initial_piles):
    explored_nodes = np.ndarray(EPISODES)
    moves_count = np.ndarray(EPISODES)
    unweighted_mean_nodes = np.ndarray(EPISODES)

    for i in tqdm(range(EPISODES)):
        game = NimGame(
            initial_piles=_initial_piles[i],
            misere=_misere[i]
        )

        agent = _misereAgent if _misere[i] else _normalAgent

        winner = game.play(
            player1=agent,
            player2=agent,
            verbose=False
        )

        """ AGENT VALIDATION """
        assert winner == NimLogic.is_p_position(_initial_piles[i], _misere[i]), "Bad agent!"

        agent.compute_mean_nodes()

        unweighted_mean_nodes[i] = agent.mean_nodes
        moves_count[i] = agent.moves_count
        explored_nodes[i] = agent.nodes_explored

    weighted_mean_nodes = explored_nodes.sum() / moves_count.sum()

    avg_explored_nodes = explored_nodes.mean()
    avg_moves_count = moves_count.mean()

    values = [weighted_mean_nodes, avg_explored_nodes, avg_moves_count]

    labels = [
        "average explored nodes per move:",
        "average explored nodes per game:",
        "average moves per game:"
    ]

    label_width = max(len(lbl) for lbl in labels)

    for lbl, val in zip(labels, values):
        print(f"{lbl:<{label_width}} {val:>10.2f}")

    return explored_nodes, moves_count, unweighted_mean_nodes

# Test running function

In [6]:
def run_tests(misere_agents, normal_agents, max_depth, pile_count, max_pile, episodes, save_path_prefix):
    print("-" * 60)
    print(f"Configuration: pile_count: {pile_count}, max_pile: {max_pile}, max_depth: {max_depth}")
    print("-" * 60)

    initial_piles = np.random.randint(1, max_pile, size=(episodes, pile_count))
    misere_modes = np.random.choice([True, False], size=episodes)

    os.makedirs(save_path_prefix, exist_ok=True)

    for agent_key in misere_agents.keys():
        filename = f"{save_path_prefix}/minimax-{agent_key}-{max_depth}-{pile_count}-{max_pile}.npz"
        if os.path.exists(filename):
            print(f"Skipping {filename}")
            continue

        misere_agent = misere_agents[agent_key]
        normal_agent = normal_agents[agent_key]

        results = test_agent(misere_agent, normal_agent, misere_modes, initial_piles)
        np.savez(filename, results)

# Loading function


In [7]:
def load_all_results(vary_pile_counts, vary_max_piles,
                     max_depth,
                     folder="../savedData/Minimax"):
    records = []
    agents = ['n','c','p','a','cp','ca','pa','cpa']
    for ag in agents:
        for hc in vary_pile_counts:
            for mh in vary_max_piles:
                fn = f"{folder}/minimax-{ag}-{max_depth}-{hc}-{mh}.npz"
                if not os.path.exists(fn):
                    continue
                exp_nodes, moves_cnt, _ = np.load(fn)['arr_0']
                records.append({
                    'agent': ag,
                    'heap_count': hc,
                    'max_heap':   mh,
                    'nodes_per_move': exp_nodes.sum() / moves_cnt.sum(),
                    'nodes_per_game': exp_nodes.mean(),
                    'moves_per_game': moves_cnt.mean()
                })
    return pd.DataFrame(records)

# Plotting function

In [8]:
def plot_results(vary_pile_counts, vary_max_piles,
                 max_depth,
                 folder="../savedData/Minimax",
                 save_folder="../savedPlots/Minimax"):
    os.makedirs(save_folder, exist_ok=True)
    df = load_all_results(vary_pile_counts, vary_max_piles, max_depth, folder)

    ordered_agents = [
        'Normal','Canonical','P-pruning','Aggressive',
        'Canonical+P-pruning','Canonical+Aggressive',
        'P-pruning+Aggressive','All optimizations'
    ]

    name_map = {
        'n':'Normal','c':'Canonical','p':'P-pruning','a':'Aggressive',
        'cp':'Canonical+P-pruning','ca':'Canonical+Aggressive',
        'pa':'P-pruning+Aggressive','cpa':'All optimizations'
    }
    df['agent_name'] = df['agent'].map(name_map)

    metrics = [
        ('nodes_per_move', 'Nodes per Move', 'Blues'),
        ('nodes_per_game', 'Nodes per Game', 'Greens'),
        ('moves_per_game', 'Moves per Game', 'Purples'),
    ]

    for col, title, cmap in metrics:
        norm = LogNorm(vmin=df[col].min(), vmax=df[col].max())

        fig, axes = plt.subplots(2, 4, figsize=(16, 8),
                                 sharex=True, sharey=True)
        axes = axes.flatten()

        for idx, (ax, agent_name) in enumerate(zip(axes, ordered_agents)):
            row, col_idx = divmod(idx, 4)
            grp = df[df['agent_name'] == agent_name]
            pivot = grp.pivot(index='heap_count',
                              columns='max_heap',
                              values=col)

            sns.heatmap(pivot, annot=True, fmt=".1f", cbar=False,
                        cmap=cmap, ax=ax,
                        xticklabels=vary_max_piles,
                        yticklabels=vary_pile_counts,
                        norm=norm,
                        annot_kws={"fontsize":16})
            ax.set_title(agent_name, fontsize=20)

            ax.set_ylabel('Heap count' if col_idx == 0 else '', fontsize=14)
            ax.set_xlabel('Maximum heap size' if row == 1 else '', fontsize=14)

            ax.tick_params(axis='both', labelsize=13)

        plt.tight_layout()
        out = f"{save_folder}/minimax_{col}_d{max_depth}.png"
        fig.savefig(out, dpi=200)
        plt.close(fig)
        print(f"Saved: {out}")


# Testing on shallow depths

In [9]:
for max_depth in [1, 2]:
    misere_agents = {
        'n': MinimaxAgent(misere=True, max_depth=max_depth),
        'c': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=True, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    normal_agents = {
        'n': MinimaxAgent(misere=False, max_depth=max_depth),
        'c': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=False, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    for pile_count in [2, 4, 8]:
        for max_count in [63, 127, 255]:
            run_tests(misere_agents, normal_agents, max_depth, pile_count, max_count, EPISODES, "../savedData/Minimax")

------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 63, max_depth: 1
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-1-2-63.npz
Skipping ../savedData/Minimax/minimax-c-1-2-63.npz
Skipping ../savedData/Minimax/minimax-p-1-2-63.npz
Skipping ../savedData/Minimax/minimax-a-1-2-63.npz
Skipping ../savedData/Minimax/minimax-cp-1-2-63.npz
Skipping ../savedData/Minimax/minimax-ca-1-2-63.npz
Skipping ../savedData/Minimax/minimax-pa-1-2-63.npz
Skipping ../savedData/Minimax/minimax-cpa-1-2-63.npz
------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 127, max_depth: 1
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-1-2-127.npz
Skipping ../savedData/Minimax/minimax-c-1-2-127.npz
Skipping ../savedData/Minimax/minimax-p-1-2-127.npz
Skipping ../savedData/Minimax/minimax-a-1-2-127.npz
Skipping ../savedDat

In [10]:
for max_depth in [1, 2]:
    plot_results(
        vary_pile_counts=[2, 4, 8],
        vary_max_piles=[63, 127, 255],
        max_depth=max_depth,
    )

Saved: ../savedPlots/Minimax/minimax_nodes_per_move_d1.png
Saved: ../savedPlots/Minimax/minimax_nodes_per_game_d1.png
Saved: ../savedPlots/Minimax/minimax_moves_per_game_d1.png
Saved: ../savedPlots/Minimax/minimax_nodes_per_move_d2.png
Saved: ../savedPlots/Minimax/minimax_nodes_per_game_d2.png
Saved: ../savedPlots/Minimax/minimax_moves_per_game_d2.png


# Max depth testing on a smaller configuration

In [11]:
misere_agents = {
    'n': MinimaxAgent(misere=True, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

normal_agents = {
    'n': MinimaxAgent(misere=False, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

for pile_count in [2, 3, 4]:
    for max_pile in [3, 4, 5]:
        run_tests(misere_agents, normal_agents, MAX_DEPTH, pile_count, max_pile, EPISODES, "../savedData/Minimax")

------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 3, max_depth: 9999
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-c-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-p-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-a-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-cp-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-ca-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-pa-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-cpa-9999-2-3.npz
------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 4, max_depth: 9999
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-9999-2-4.npz
Skipping ../savedData/Minimax/minimax-c-9999-2-4.npz
Skipping ../savedData/Minimax/minimax-p-9999-2-4.npz
Skipping ../savedData/Minimax/minimax-a-9999-2-4.n

In [12]:
plot_results(
    vary_pile_counts=[2, 3, 4],
    vary_max_piles=[3, 4, 5],
    max_depth=MAX_DEPTH
)


Saved: ../savedPlots/Minimax/minimax_nodes_per_move_d9999.png
Saved: ../savedPlots/Minimax/minimax_nodes_per_game_d9999.png
Saved: ../savedPlots/Minimax/minimax_moves_per_game_d9999.png
